In [ ]:
!pip install transformers -U
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-hor5r1d0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-hor5r1d0
  Resolved https://github.com/huggingface/transformers.git to commit 8e164c5400b7b413c7b8fb32e35132001effc970
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel fo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Train_toxic-comment.csv')
data.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = model.to('cuda')

In [ ]:
sample_data = ['I am eating', 'I am playing']
tokenizer(sample_data, padding = True, max_length = 512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [ ]:
x = list(data['comment_text'])
y = list(data['toxic'])
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, stratify=y)
X_train_tokenized = tokenizer(x_train, padding = True, truncation= True, max_length=512)
X_val_tokenized = tokenizer(x_val, padding = True, truncation= True, max_length=512)

In [ ]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = IMDbDataset(X_train_tokenized, y_train)
val_dataset = IMDbDataset(X_val_tokenized, y_val)

In [ ]:
train_dataset[5]

{'input_ids': tensor([  101,  3433,  2092,  1010,  4283,  2005,  3228,  2033,  1037,  2345,
          5432,  1012,  2004,  5156,  1010,  1045,  2131,  2053, 12284,  2005,
          9186,  4790,  2006,  2023,  2609,  1010,  2053,  2616,  1997, 15846,
          2005,  1996,  2847,  2008,  1045,  5247,  3554,  3158,  9305,  2964,
          1025,  2074, 10231,  1012, 10231,  2005,  1996,  1014,  1012,  1015,
          1003,  1997,  2026, 10086,  2015,  2008,  2071,  2022,  2649,  2004,
          2512,  8663,  3367,  6820, 15277,  1012,  2126,  2000,  2175,  1010,
         22822,  2239,  1012,  2562,  2039,  1996,  2204,  2147,  1012,  1011,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
def compute_metrics(p):
  print(type(p))
  pred, labels = p
  pred = np.argmax(pred, axis = 1)
  accuracy = accuracy_score(y_true = labels, y_pred = pred)
  recall = recall_score(y_true = labels, y_pred = pred)
  precesion = precision_score(y_true= labels, y_pred = pred)
  f1 = f1_score(y_true = labels, y_pred = pred)
  return {"accuracy": accuracy, "precesion": precesion, "recall": recall, "f1": f1}

In [ ]:

args = TrainingArguments(
    output_dir="output",
    num_train_epochs= 1,
    per_device_train_batch_size=8
)
trainer = Trainer(
    model= model,
    args= args,
    train_dataset = train_dataset,
    eval_dataset= val_dataset,
    compute_metrics = compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.193200
1000,0.146400


TrainOutput(global_step=1000, training_loss=0.16978206634521484, metrics={'train_runtime': 725.6748, 'train_samples_per_second': 11.024, 'train_steps_per_second': 1.378, 'total_flos': 2104888442880000.0, 'train_loss': 0.16978206634521484, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.1169738918542862,
 'eval_accuracy': 0.961,
 'eval_precesion': 0.7871287128712872,
 'eval_recall': 0.8195876288659794,
 'eval_f1': 0.803030303030303,
 'eval_runtime': 63.209,
 'eval_samples_per_second': 31.641,
 'eval_steps_per_second': 3.955,
 'epoch': 1.0}

In [ ]:
text = "That was a good point"
#text = "Go to hell, Stupid"
inputs = tokenizer(text, padding= True, truncation= True, return_tensors= 'pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 3.0878, -2.8092]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.9973, 0.0027]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.99725986, 0.00274015]], dtype=float32)

In [ ]:
trainer.save_model('CustomModel')
trainer.save_model("/content/drive/MyDrive/CustomModel")

In [ ]:
model_2 = BertForSequenceClassification.from_pretrained("CustomModel")
model_2.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# text = "That was good point"
text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.0047047 , 0.99529535]], dtype=float32)

In [ ]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/CustomModel")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub('BadisNahdi/FineTuneBERT')

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BadisNahdi/FineTuneBERT/commit/e6d6421c1b61c9c5774dcd82a2a313a743a42d6b', commit_message='Upload BertForSequenceClassification', commit_description='', oid='e6d6421c1b61c9c5774dcd82a2a313a743a42d6b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
load_model = BertForSequenceClassification.from_pretrained('BadisNahdi/FineTuneBERT')

In [ ]:
load_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,